In [ ]:
import numpy as np
import nltk
import string
import random

f = open('/chatbot.txt', 'r', errors='ignore')


raw_doc = f.read()
raw_doc = raw_doc.lower()    # Converts text to lowercase

nltk.download('punkt')
nltk.download('punkt_tab')   # <-- required for new NLTK versions
nltk.download('wordnet')

sent_tokens = nltk.sent_tokenize(raw_doc)
word_tokens = nltk.word_tokenize(raw_doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
sent_tokens[:2]

['\n\nwikipediathe free encyclopedia\nsearch wikipedia\nsearch\ndonate\ncreate account\nlog in\nbanner logo\t\njoin wikipedia asian month this november and december!',
 'contribute in wikipedia asian month and get a postcard!']

In [ ]:
word_tokens[:2]

['wikipediathe', 'free']

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()

# WordNet is a semantically-oriented dictionary of English included in NLTK.

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [ ]:
GREET_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREET_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greet(sentence):
    for word in sentence.split():
        if word.lower() in GREET_INPUTS:
            return random.choice(GREET_RESPONSES)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(user_response):
    robo1_response = ''

    # Create TF-IDF vectorizer
    TfidVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

    # Fit TF-IDF on the sentence tokens
    tfidf = TfidVec.fit_transform(sent_tokens)

    # Compute cosine similarity
    vals = cosine_similarity(tfidf[-1], tfidf)

    # Pick the second-best similarity index
    idx = vals.argsort()[0][-2]

    # Flatten similarity scores
    flat = vals.flatten()
    flat.sort()

    req_tfidf = flat[-2]

    # If no similarity found
    if req_tfidf == 0:
        robo1_response = "I am sorry! I don't understand you"
        return robo1_response
    else:
        robo1_response = sent_tokens[idx]
        return robo1_response



In [ ]:
flag = True
print("BOT: My name is Stark. Let's have a conversation! Also, if you want to exit any time, just type Bye!")

while flag == True:
    user_response = input()
    user_response = user_response.lower()

    if user_response != 'bye':
        if user_response == 'thanks' or user_response == 'thank you':
            flag = False
            print("BOT: You are welcome..")

        else:
            if greet(user_response) != None:
                print("BOT: " + greet(user_response))

            else:
                sent_tokens.append(user_response)
                word_tokens = word_tokens + nltk.word_tokenize(user_response)
                final_words = list(set(word_tokens))

                print("BOT: ", end="")
                print(response(user_response))

                sent_tokens.remove(user_response)

    else:
        flag = False
        print("BOT: Goodbye! Take care <3")


BOT: My name is Stark. Let's have a conversation! Also, if you want to exit any time, just type Bye!
hii
BOT: I am sorry! I don't understand you
hi
BOT: hi there


KeyboardInterrupt: Interrupted by user